# Quickdraw Doodle
## Train

In [1]:
image_size = 96
batch_size = 256
model_path = 'model.json'
weights_path = 'model.h5'

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lib.preprocessing import json_to_array, image_generator
from lib.data import read_dataset_folder, get_classes, get_label_encoder_y, image_generator

### Import data set

In [4]:
%%time
dataset_train = read_dataset_folder('train_simplified/set_14/')
dataset_test = read_dataset_folder('test_simplified/set_1/', frac=0.1)
dataset_train['drawing'] = dataset_train['drawing'].apply(json_to_array)
dataset_test['drawing'] = dataset_test['drawing'].apply(json_to_array)
print('Training set =', len(dataset_train))
print('Test set =', len(dataset_test))

Training set = 438123
Test set = 34000
CPU times: user 21 s, sys: 1.33 s, total: 22.3 s
Wall time: 22.4 s


### Prepare data

In [5]:
train_generator = image_generator(dataset_train, include_y=True)
test_generator = image_generator(dataset_test, include_y=True)

In [6]:
label_y = dataset_train['word'].values
from sklearn.preprocessing import OneHotEncoder
labelEncoder_y = get_label_encoder_y()
label_y = labelEncoder_y.transform(label_y).reshape(-1, 1)
oneHotEncoder = OneHotEncoder(categorical_features=[0])
oneHotEncoder.fit(label_y)

OneHotEncoder(categorical_features=[0], dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

### Create model

In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [10]:
classifier = InceptionResNetV2(input_shape=(image_size, image_size, 1), weights=None, classes=len(get_classes()))
classifier.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Load model

In [7]:
model_file = open(model_path)
model_json = model_file.read()

In [8]:
from keras.models import model_from_json
classifier = model_from_json(model_json)
classifier.load_weights(weights_path)
classifier.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

Using TensorFlow backend.


### Training

In [ ]:
classifier.fit_generator(train_generator, steps_per_epoch=int(math.ceil(len(dataset_train)/batch_size)), epochs=1,
                         validation_data=test_generator, validation_steps=int(math.ceil(len(dataset_test)/batch_size)))

### Save model

In [10]:
%%time
model_json = classifier.to_json()
with open(model_path, 'w') as model_file:
    model_file.write(model_json)
classifier.save_weights(weights_path)

CPU times: user 3min 30s, sys: 12 ms, total: 3min 30s
Wall time: 3min 30s
